In [1]:
import os
import json
from docx import Document
from io import StringIO, BytesIO
import re

import pandas as pd
import json
import spacy
from nltk.corpus import stopwords

from gensim.models import LdaModel
from gensim.models.wrappers import LdaMallet
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim import matutils, models
from gensim.models import CoherenceModel
import pyLDAvis.gensim

from docx import Document
from io import StringIO, BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'font.size': 14, 'lines.linewidth': 3})

In [2]:
notebook_dir = os.getcwd()
sop_df = pd.read_csv('../data/interim/sop_topics_2020-05-26T01_34_46.csv', 
                    converters = {'sop': eval})
sop_df = sop_df.rename(columns = {'topic_id': 'group_id'})
sop_df

,type,juri,role,sop,filename,group_id
0,1033,AB,call taker,"[Create a call, Questions, Are there weapons i...",AB - 1033 - Officer in trouble.docx,7
1,1033,BI,call taker,"[Create a call, Questions, Are there weapons i...",BI - 1033 - Officer in trouble.docx,7
2,1033,BU,call taker,"[Create a call, Questions, Are there weapons i...",BU - 1033 - Officer in trouble.docx,7
3,1033,DE,call taker,"[Create a call, Questions, Are there weapons i...",DE - 1033 - Officer in trouble.docx,7
4,1033,DFPF,call taker,"[Create a call, Questions, Are there weapons i...",DFPF - 1033 - Officer in trouble.docx,7
...,...,...,...,...,...,...
2924,DNA,WV,dispatcher,[This file type is not used. ],WV - DNA - DNA collection.docx,1
2925,DNA,NW,dispatcher,[Person attends front counter or member reques...,NW - DNA - DNA collection .docx,3
2926,DNA,PO,dispatcher,[Assign a member],PO - DNA - DNA collection.docx,3
2927,DNA,RI,dispatcher,"[Treat file as a Priority 2, Assign a DNA cert...",RI - DNA - DNA collection.docx,3


In [3]:
sop_df['type'].unique()

array(['1033', 'DOMI', 'FOUNDP', 'HAZARD', 'IMPAIR', '911', 'INTELL',
       'JUMPER', 'MAND', 'MISCH', 'MISCHI', 'MVI', 'MVIHR', 'MVIINJ',
       'OCC', 'PANHA', 'PROP', 'PURSUE', 'RECVEH', 'SCREAM', 'SHOTS',
       'SIP', 'SUDDEN', 'SUICID', 'SUSPC', 'SUSPP', 'THEFTV', 'THREATS',
       'TRAFF', 'TRAFFS', 'WEAPON', 'DRUGS', 'INSEC', 'ANIMAL', 'ASSOA',
       'ABANDV', 'ARREST', 'ARSON', 'MVIPOL', 'PARK', 'ANNOY', 'OVERD',
       'NOK', 'BREACH', 'PROWL', 'ASLT', 'MISSIP', 'MISSIC', 'ASLTI',
       'ASLTSX', 'PROST', 'ALARMS', 'ASSGP', 'THEFT', 'AB911 CELL',
       'WARRAN', 'UNWANT', 'AB911 LANDLINE', 'ABDUC', 'THEFTI', 'SUSPV',
       'ROBB', 'AIREM', 'ALARM', 'STALK', 'SHOPL', 'ALARMH', 'ROBBI',
       'MARINE', 'YOUTH', 'LIQUOR', 'COUNTERFEIT', 'EXPLOS', 'EXTORT',
       'BNEI', 'DOMRPT', 'FIGHT', 'FIREAR', 'BOMB', 'BNE', 'FRAUD',
       'DISTB', 'HARASS', 'DEMON', 'ASSPFA', 'HOMEIN', 'HOSTAG', 'BAIT',
       'CHECK', 'BYLAW', 'INDEC', 'KPEACE', 'INDUST', 'ASSMHA', 'SPAT',
       

In [5]:
def show_result(df, juris):
    for text in df[df['juri'].isin(juris)]['sop']:
        print(text)
        print('======')

In [6]:
misch_call = sop_df[ (sop_df['type'] == 'MISCH') & (sop_df['role'] == 'call taker') ]
misch_call

,type,juri,role,sop,filename,group_id
285,MISCH,DFPF,call taker,[Refer caller to Police of Jurisdiction],DFPF - MISCH - Mischief.docx,0
286,MISCH,VA,call taker,[If the mischief involves any of the following...,VA - MISCH - Mischief.docx,0
287,MISCH,BI,call taker,"[Create a call, Mischief to auto, Create a cal...",BI - MISCH - Mischief.docx,2
288,MISCH,DE,call taker,"[Create a call, Mischief to auto, Create a cal...",DE - MISCH - Mischief.docx,2
289,MISCH,NW,call taker,"[Create a call, Mischief to auto, Create a cal...",NW - MISCH - Mischief.docx,2
290,MISCH,PO,call taker,"[Create a call, Mischief to auto, Create a cal...",PO - MISCH - Mischief.docx,2
291,MISCH,RM,call taker,"[Create a call, Mischief to auto, Create a cal...",RM - MISCH - Mischief.docx,2
292,MISCH,SC,call taker,"[Create a call, Mischief to auto, Create a cal...",SC - MISCH - Mischief.docx,2
293,MISCH,SQ,call taker,"[Create a call, Mischief to auto, Create a cal...",SQ - MISCH - Mischief.docx,2
294,MISCH,SX,call taker,"[Create a call, Firecracker / Fireworks use , ...",SX - MISCH - Mischief.docx,2


### Grouping that makes sense

In [7]:
juris = {'AB', 'BU', 'RI'}
show_result(misch_call, juris)

['If the mischief involves any of the following, refer to the appropriate section', ' ', 'For all other instances', 'Transfer to a Report Agent ', 'Firecracker / Fireworks use ', 'If any injuries ', 'Refer to the  or  event type SOP', 'In all other instances', 'Create a call ', 'Graffiti involving hatred, racial, ethnic bias, religious or other recognized group bias', 'Create a call']
['If the mischief involves any of the following, refer to the appropriate section', ' ', 'For all other time delayed reports of mischief that do not involve any element of a break and enter', ' Transfer to a Report Agent ', 'Firecracker / Fireworks use ', 'If any injuries ', 'Refer to the ASLT or ASLTI event type SOP', 'In all other instances', 'Create a call ', 'Graffiti involving hatred, racial, ethnic bias, religious or other recognized group bias', 'Create a call']
['If the mischief involves any of the following, refer to the appropriate section', 'For all other time delayed reports of mischief that d

In [16]:
juris = {'DE', 'BI', 'WV'}
show_result(misch_call, juris)

['Create a call', 'Mischief to auto', 'Create a call', 'Provide file number', 'Firecracker / Fireworks use ', 'If any injuries see the ASLT or ASLTI event type SOP', 'Otherwise, create a call']
['Create a call', 'Mischief to auto', 'Create a call', 'Provide file number', 'Firecracker / Fireworks use ', 'If any injuries see the ASLT or ASLTI event type SOP', 'Otherwise, create a call']
['Create a call', 'Mischief to auto', 'Create a call', 'Provide file number', 'Firecracker / Fireworks use ', 'If any injuries see the ASLT or ASLTI event type SOP', 'Otherwise, create a call']


### Grouping that might not make sense

In [9]:
juris = {'WP', 'WV'}
show_result(misch_call, juris)

['Create a call', 'Mischief to auto in Pemberton', 'If the vehicle can be driven', 'Advise the caller to attend the office. ', 'If the vehicle cannot be driven', 'Advise the caller to call the office during business hours.', 'Mischief to auto in Whistler', 'Create a call', 'Provide file number', 'Firecracker / Fireworks use ', 'If any injuries see the ASLT or ASLTI event type SOP', 'Otherwise, create a call']
['Create a call', 'Mischief to auto', 'Create a call', 'Provide file number', 'Firecracker / Fireworks use ', 'If any injuries see the ASLT or ASLTI event type SOP', 'Otherwise, create a call']


### Hyper link in the text

In [10]:
juris = {'VA'}
show_result(misch_call, juris)

['If the mischief involves any of the following, refer to the appropriate section', 'For all other time delayed reports of mischief that do not involve any element of a break and enter', 'Advise caller that their report can be made online at  or you can send them an e-mail with the link', 'If the caller requests that you send them a link', 'Enter the callers e-mail address in the ', 'If caller wishes to make report on the phone', 'Transfer to a Report Agent', 'Create a call', 'City property', 'Refer the caller to 3-1-1 to take the report', 'Advise caller that 3-1-1 will make a police report if required', 'Firecracker / Fireworks use ', 'If any injuries ', 'Refer to ASLT or ASLTI event type SOP', 'If no injuries', 'Create a call ', 'Garbage dumping ', 'If potentially hazardous material', 'Refer to the  event SOP', 'If on public property', 'Refer caller to 3-1-1', 'If on private property and there is video surveillance/witness that captures the suspect and offence in its entirety', 'Crea

In [13]:
misch_dispatch = sop_df[ (sop_df['type'] == 'MISCH') & (sop_df['role'] == 'dispatcher') ]
misch_dispatch

,type,juri,role,sop,filename,group_id
301,MISCH,AB,dispatcher,[Property (includes requeued calls and value >...,AB - MISCH - Mischief.docx,1
302,MISCH,BU,dispatcher,"[Property (includes known suspect, value > $50...",BU - MISCH - Mischief.docx,1
303,MISCH,RI,dispatcher,"[Firecracker / Fireworks use, General broadcas...",RI - MISCH - Mischief .docx,1
304,MISCH,SX,dispatcher,"[If a member is on shift, Assign a member, If ...",SX - MISCH - Mischief.docx,1
305,MISCH,BI,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",BI - MISCH - Mischief.docx,2
306,MISCH,DE,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",DE - MISCH - Mischief.docx,2
307,MISCH,NW,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",NW - MISCH - Mischief.docx,2
308,MISCH,PO,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",PO - MISCH - Mischief.docx,2
309,MISCH,RM,dispatcher,[Assign a member],RM - MISCH - Mischief.docx,2
310,MISCH,SC,dispatcher,"[Assign a member, Mischief to auto, Assign a m...",SC - MISCH - Mischief.docx,2
